## Google Scholar Crawler
- **DISCLAIMER**: usually Google block the execturion of this script after retrieving between 400 and 500 entries (40 and 50 pages).
- **TODO**: add selenium and add a manual step for opening a url and showing that we are NOT a robot

In [ ]:
keyword_raw = "honeypot"

In [ ]:
import cfscrape
from lxml import etree
import pandas as pd
import urllib
import re
from random import randint
from time import sleep

In [ ]:
header = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9',
    'Accept-Encoding': 'gzip, deflate, sdch',
    'Accept-Language' : 'nl-NL,nl;q=0.8,en-US;q=0.6,en;q=0.4',
    'Cache-Control' : 'max-age=0',
    'Connection': 'keep-alive',
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/43.0.2357.81 Safari/537.36'
    }

In [ ]:
page = 1 
df = pd.DataFrame()
    
while True: 
    start_page = (page - 1) * 10
    #################################################################################
    keyword = urllib.parse.quote_plus("\""+keyword_raw+"\"")
    url="https://scholar.google.nl/scholar?start="+str(start_page)+"&q="+keyword+"&hl=nl&as_sdt=0,5"
    print(page, url)
    #################################################################################
    scraper = cfscrape.create_scraper()
    scraped_html=scraper.get(url,headers=header).content
    # print(scraped_html.decode("utf-8") )
    html = etree.HTML(scraped_html)
    #################################################################################    
    elements = html.xpath("//div[@class='gs_ri']")
    #################################################################################
    if len(elements) == 0:
        print("The code stopped for some reason!! Possibly Google blocked us!")
        print("\""+(''.join(html.xpath("//div[@id='infoDiv']/text()")))+"\"")
        break
    #################################################################################
    for element in elements:
        #################################################################################
        if len(element.xpath("h3[@class='gs_rt']/a/text()")) > 1:
            title = (keyword_raw+" ").join(element.xpath("h3[@class='gs_rt']/a/text()"))
        elif len(element.xpath("h3[@class='gs_rt']/a/text()")) == 1:
            title = element.xpath("h3[@class='gs_rt']/a/text()")[0]
        else:
            title = ""
        #################################################################################    
        link  = ''.join(element.xpath("h3[@class='gs_rt']/a/@href"))
        #################################################################################    
        abstract = ''.join(element.xpath("div[@class='gs_rs']/text()")).replace('\n','')
        #################################################################################
        reference = ''.join(element.xpath("div[@class='gs_a']/text()"))
        #################################################################################
        try:
            year = re.search(r"\d{4}", reference)[0]
        except:
            year = ""
        #################################################################################   
        df = df.append({'year':year,'title':title,'abstract':abstract,'reference':reference,'link':link},ignore_index=True)
    #################################################################################   
    print("sleeping...")
    sleep(randint(6,18))
    #################################################################################   
    page = page + 1       

## Exporting the output to a csv file

In [ ]:
df.to_csv('honeypot_googlescholar_query_honeypot_20190722.csv')

<h1 align='center'> !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!<br>OFFLINE ANALYSIS STARTS HERE</h1>

In [1]:
import pandas as pd
pd.options.display.max_colwidth = 100
from urllib.parse import urlparse

In [2]:
#Reading again the file in case it is a offline analysis
df = pd.read_csv('honeypot_googlescholar_query_honeypot_20190722.csv',index_col=0).dropna()
df.head()

,abstract,link,reference,title,year
0,A is a closely monitored network decoy serving several purposes: it can distract adversaries fro...,https://www.usenix.org/event/sec04/tech/full_papers/provos/provos_html,"- USENIX Security Symposium, 2004 - usenix.org",A Virtual honeypot Framework.,2004
1,"Spammers continually scan the Internet for open proxy relays: by using these open relays, they c...",https://ieeexplore.ieee.org/abstract/document/1264861/,"N Krawetz - IEEE Security & Privacy, 2004 - ieeexplore.ieee.org",Anti-honeypot technology,2004
2,Honeypots are closely monitored network decoys serving several purposes: they can distract adver...,https://www.researchgate.net/profile/Niels_Provos/publication/250395424_Honeyd_A_Virtual_Honeypo...,"- 10th DFN-CERT Workshop, Hamburg, Germany, 2003 - researchgate.net",Honeyd-a virtual honeypot daemon,2003
3,"The number of client-side attacks has grown significantly in the past few years, shifting focus ...",https://www.usenix.org/legacy/event/leet09/tech/full_papers/nazario/nazario.pdf,"- LEET, 2009 - usenix.org",PhoneyC: A Virtual Client honeypot .,2009
4,"Some attacks on honeypots are very frequent and repetitive. In addition, such repetitive attacks...",ftp://ftp.mirror.ac.za/www.honeynet.org/papers/individual/AusCERT_fullpaper_BIS.pdf,"F Pouget, - AusCERT Asia Pacific Information Technology …, 2004 - mirror.ac.za",-based forensics,2004


In [3]:
print("TOTAL ENTRIES:",len(df))

TOTAL ENTRIES: 393


## Distribution of source of academic papers (digital libraries)!

In [9]:
digital_libraries_distribution.head()

,counts,percentage
ieeexplore.ieee.org,141,35.877863
link.springer.com,38,9.669211
en.cnki.com.cn,22,5.597964
pdfs.semanticscholar.org,21,5.343511
patents.google.com,20,5.089059


In [5]:
digital_libraries = df['link'].dropna().apply(lambda x: urlparse(x)[1].replace("www.",""))

digital_libraries_distribution = pd.concat([digital_libraries.value_counts(), 
                                            digital_libraries.value_counts(normalize=True).mul(100)],axis=1, keys=('counts','percentage'))

print("Distinct Source of Information:",len(digital_libraries_distribution))
print()
print("Frequency of Source of Information:")
print(digital_libraries_distribution)

Distinct Source of Information: 74

Frequency of Source of Information:
                                       counts  percentage
ieeexplore.ieee.org                       141   35.877863
link.springer.com                          38    9.669211
en.cnki.com.cn                             22    5.597964
pdfs.semanticscholar.org                   21    5.343511
patents.google.com                         20    5.089059
dl.acm.org                                 17    4.325700
citeseerx.ist.psu.edu                      15    3.816794
sciencedirect.com                           9    2.290076
researchgate.net                            9    2.290076
ro.ecu.edu.au                               5    1.272265
academia.edu                                5    1.272265
usenix.org                                  5    1.272265
cqvip.com                                   5    1.272265
onlinelibrary.wiley.com                     5    1.272265
tandfonline.com                             4    1.017812


The most important source of information are: 
- ieeexplore.ieee.org
- dl.acm.org
- link.springer.com
- pdfs.semanticscholar.org
- sciencedirect.com
- researchgate.net

Of course the RFCs of IETF are also very important:
- rfc-editor.org 
- tools.ietf.org
- datatracker.ietf.org

## Exporting information to csv files based on the source of information

## Crawling abstracts from IEEE

In [ ]:
df[df['link'].str.contains("ieeexplore.ieee.org")].to_csv("honeypot_ieee.csv")

In [ ]:
for link in df[df['link'].str.contains("ieeexplore.ieee.org")]['link']:
    print(link)
    #################################################################################
    scraper = cfscrape.create_scraper()
    scraped_html=scraper.get(link,headers=header).content
    # print(scraped_html.decode("utf-8") )
    html = etree.HTML(scraped_html)
    #################################################################################    
    elements = html.xpath("//script[@type='text/javascript']/text()")
    abstract = elements[2].split('"abstract":"')[2].split('","')[0]
    print(abstract)  

In [ ]:
df[df['link'].str.contains("link.springer.com")].to_csv("flowspec_springer.csv")

In [ ]:
df[df['link'].str.contains("dl.acm.org")].to_csv("flowspec_acm.csv")

## Lessons learned:
- remove papers that have in the title or description/abstract the list of words: [rsvp, qos, quality of service, mpls, intserv, diffserv, multicast]